# Project: Building Fast Queries on a CSV

## Context 

We will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory.

We will use the laptops.csv file as our inventory. This CSV file was adapted from the [Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices). We changed the IDs and made the prices integers.

##  Import the Inventory Data

In [46]:
import csv

# read in file
with open('laptops.csv', encoding='utf-8') as file:
    opened_file = csv.reader(file)
    inventory = list(opened_file)
    header = inventory[0]
    rows = inventory[1:]
    
# display the header
print(header, "\n")

# display the first five rows
for row in rows[:6]:
    print(row, "\n")


['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price'] 

['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'] 

['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'] 

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'] 

['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'] 

['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '2

## Creating an Inventory Class 

In [47]:
class inventory():
    def __init__(self, csv_file):
        with open(csv_file, encoding='utf-8') as file:
            opened_file = csv.reader(file)
            inventory = list(opened_file)
            self.header = inventory[0]
            self.rows = inventory[1:]
            
            for row in self.rows:
                price = int(row[-1])
                row[-1] = price
    
laptop_inv = inventory('laptops.csv')

# display the attributes (header, first 3 rows) of the inventory to check if it works
print(laptop_inv.header, "\n")

for row in laptop_inv.rows[:3]:
    print(row, "\n")

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price'] 

['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339] 

['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', 898] 

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575] 



## Finding a Laptop from the Inventory Based on the ID 

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

For this, we will write a function named get_laptop_from_id(). This function will take as argument the identifier of the laptop and return the full row of the laptop with that id.

In [48]:
# create a updated inventory class with the get_laptop_from_id() method
class inventory():
    def __init__(self, csv_file):
        with open(csv_file, encoding='utf-8') as file:
            opened_file = csv.reader(file)
            inventory = list(opened_file)
            self.header = inventory[0]
            self.rows = inventory[1:]
            
            for row in self.rows:
                price = int(row[-1])
                row[-1] = price

    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            product_id = row[0]
            
            if laptop_id == product_id:
                return row
            
        return None
    
# test new method
laptop = inventory('laptops.csv')

# if laptop exists
print("Existing Laptop ID")
result = laptop.get_laptop_from_id('3362737')
print("Result:", result, "\n")

# if laptop id does not exists print nothing
print("Non Existent Laptop ID")
result = laptop.get_laptop_from_id('3362736')
print("Result:", result, "\n")

Existing Laptop ID
Result: ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575] 

Non Existent Laptop ID
Result: None 



<b>Comment:<\b> The time complexity of this method will take O(n), where n is the number of rows of the csv file (excluding header). Now, we should create a faster way of retrieval of the laptop data. To get rid of the for loop, we can try to store the data into a dictionary where the laptop id is the key and the value is the data.

## Improving The Laptop ID Lookups 

In [49]:
# the updated inventory class
class inventory():
    def __init__(self, csv_file):
        self.id_to_row = {}
        
        with open(csv_file, encoding='utf-8') as file:
            opened_file = csv.reader(file)
            inventory = list(opened_file)
            self.header = inventory[0]
            self.rows = inventory[1:]
            
            for row in self.rows:
                price = int(row[-1])
                row[-1] = price
                
                # store data in dictionary
                laptop_id = row[0]
                self.id_to_row[laptop_id] = row

    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            product_id = row[0]
            
            if laptop_id == product_id:
                return row
            
        return None
    
    def get_laptop_from_id_faster(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None
        
# test new method
laptop = inventory('laptops.csv')

# if laptop exists
print("Existing Laptop ID")
result = laptop.get_laptop_from_id_faster('3362737')
print("Result:", result, "\n")

# if laptop id does not exists print nothing
print("Non Existent Laptop ID")
result = laptop.get_laptop_from_id_faster('3362736')
print("Result:", result, "\n")

Existing Laptop ID
Result: ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575] 

Non Existent Laptop ID
Result: None 



<b>Comment</b>: Here, the new method of id retrieval has a time complexity of O(1) as all we are checking is if the laptop_id exists as a key in the dictionary we created.

## Comparing the Performance of Both Methods 

In [50]:
import time, random

# generate 10,000 random values
ids = [random.randint(1000000, 9999999) for _ in range(10000)]

# instantiate new object from the inventory class
laptop = inventory('laptops.csv')

# get performance of first method
total_time_no_dict = 0

for laptop_id in ids:
    start = time.time()
    result = laptop.get_laptop_from_id(laptop_id)
    end = time.time()
    total_time_no_dict += end - start

# get performance of second method    
total_time_dict = 0

for laptop_id in ids:
    start = time.time()
    result = laptop.get_laptop_from_id_faster(laptop_id)
    end = time.time()
    total_time_dict += end - start
    
# display the total time for both methods
print("For Loop Performance:", total_time_no_dict)
print("Dictionary Performance:", total_time_dict)

For Loop Performance: 1.6590468883514404
Dictionary Performance: 0.00624394416809082


<b>Comment:</b> Clearly, the performance of the dictionary retrieval method is faster as postulated earlier.

## Scenario: Two Laptop Promotions

Sometimes, your store offers a promotion where you give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

For example, imagine that there are only three laptops in inventory:

The prices of these laptops are \$1,339, \$898, and \$575. Say we offered a gift card of \$2,500. Since a customer can buy, at most, two laptops with a gift card, the maximum they can spend is \$2,237 (\$1,339 plus \$898). Therefore, they might feel cheated because, no matter how they spend their gift card, they cannot spend the full \$2,500.

You don't want to make a customer feel cheated, so whenever you issue a gift card, you want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for D dollars, you want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

We should write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

In [51]:
# the updated inventory class
class inventory():
    def __init__(self, csv_file):
        self.id_to_row = {}
        
        with open(csv_file, encoding='utf-8') as file:
            opened_file = csv.reader(file)
            inventory = list(opened_file)
            self.header = inventory[0]
            self.rows = inventory[1:]
            
            for row in self.rows:
                price = int(row[-1])
                row[-1] = price
                
                # store data in dictionary
                laptop_id = row[0]
                self.id_to_row[laptop_id] = row

    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            product_id = row[0]
            
            if laptop_id == product_id:
                return row
            
        return None
    
    def get_laptop_from_id_faster(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None
    
    def check_promotion_dollars(self, dollars):
        # check if a laptop cost the exact dollar amount
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
        
        # check if pairs of laptops add up to the dollar amount
        amount = 0
        for row in self.rows:
            a_price = row[-1]
            for row in self.rows:
                b_price = row[-1]
                if (a_price + b_price) == dollars:
                    return True
            
        # return false if it is impossible to spend the specified dollar amount
        return False
        
# test new method with two scenarios: solution (laptops found) vs no solution
laptop = inventory('laptops.csv')
result_a = laptop.check_promotion_dollars(1000)
result_b = laptop.check_promotion_dollars(442)

print("Laptops found that equal exactly $1000:", result_a) # solution
print("Laptops found that equal exactly $442:", result_b) # no solution

Laptops found that equal exactly $1000: True
Laptops found that equal exactly $442: False


## Optimizing Laptop Promotion Method 

Now, we should optimize our check_promotion_dollars as the for loop has a time complexity of O(n^2).

In [52]:
# the updated inventory class
class inventory():
    def __init__(self, csv_file):
        self.id_to_row = {}
        self.prices = set()
        
        with open(csv_file, encoding='utf-8') as file:
            opened_file = csv.reader(file)
            inventory = list(opened_file)
            self.header = inventory[0]
            self.rows = inventory[1:]
            
            for row in self.rows:
                price = int(row[-1])
                row[-1] = price
                
                # store data in dictionary
                laptop_id = row[0]
                self.id_to_row[laptop_id] = row
                self.prices.add(price)
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            product_id = row[0]
            
            if laptop_id == product_id:
                return row
            
        return None
    
    def get_laptop_from_id_faster(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None
    
    def check_promotion_dollars(self, dollars):
        # check if a laptop cost the exact dollar amount
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
        
        # check if pairs of laptops add up to the dollar amount
        amount = 0
        for row in self.rows:
            a_price = row[-1]
            for row in self.rows:
                b_price = row[-1]
                if (a_price + b_price) == dollars:
                    return True
            
        # return false if it is impossible to spend the specified dollar amount
        return False
    
    def check_promotion_dollars_faster(self, dollars):
        if dollars in self.prices:
            return True
        
        for price in self.prices:
            b_price = dollars - price
            if b_price in self.prices:
                return True
            
        return False
    
# test new method with two scenarios: solution (laptops found) vs no solution
laptop = inventory('laptops.csv')
result_a = laptop.check_promotion_dollars_faster(1000)
result_b = laptop.check_promotion_dollars_faster(442)

print("Laptops found that equal exactly $1000:", result_a) # solution
print("Laptops found that equal exactly $442:", result_b) # no solution    

Laptops found that equal exactly $1000: True
Laptops found that equal exactly $442: False


## Comparing Laptop Promotion Methods 

Now, let us compare the methods we just implemented to check the performance of each.

In [53]:
# generate 100 random values between 100 and 5000
prices = [random.randint(100,5000) for _ in range(100)]

# create instance of inventory
laptops = inventory('laptops.csv')

# check time of first method
total_time_no_set = 0

for price in prices:
    start = time.time()
    result = laptops.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += (end - start)
    
# check time of second method
total_time_set = 0

for price in prices:
    start = time.time()
    result = laptops.check_promotion_dollars_faster(price)
    end = time.time()
    total_time_set += (end - start)
        
# display results
print("Performance without using sets:", total_time_no_set)
print("Performance using sets:", total_time_set)

Performance without using sets: 2.8923263549804688
Performance using sets: 0.001329660415649414


<b>Comment</b> Clearly, using sets are much better for performance than the double for loops we used previously. This correlates with their respective time complexities of each method with were O(n^2) and O(n).

## Finding Laptops Within a Budget

In this project, we are going to leverage and extend the binary search algorithm to help a customer find all laptops that fall within their budget.

More formally, we want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price it at most D.

If we sort all laptops by price, we can use binary search to identify the first laptop in the sorted list with a price larger than D. We need to make sure that our binary search finds the first one on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than D dollars:

In [60]:
# the updated inventory class
class inventory():
    def __init__(self, csv_file):
        self.id_to_row = {}
        self.prices = set()
        
        with open(csv_file, encoding='utf-8') as file:
            opened_file = csv.reader(file)
            inventory = list(opened_file)
            self.header = inventory[0]
            self.rows = inventory[1:]
            
            for row in self.rows:
                price = int(row[-1])
                row[-1] = price
                
                # store data in dictionary
                laptop_id = row[0]
                self.id_to_row[laptop_id] = row
                self.prices.add(price)
        
        self.rows_by_price = sorted(self.rows, key=lambda row: row[-1])
          
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            product_id = row[0]
            
            if laptop_id == product_id:
                return row
            
        return None
    
    def get_laptop_from_id_faster(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None
    
    def check_promotion_dollars(self, dollars):
        # check if a laptop cost the exact dollar amount
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
        
        # check if pairs of laptops add up to the dollar amount
        amount = 0
        for row in self.rows:
            a_price = row[-1]
            for row in self.rows:
                b_price = row[-1]
                if (a_price + b_price) == dollars:
                    return True
            
        # return false if it is impossible to spend the specified dollar amount
        return False
    
    def check_promotion_dollars_faster(self, dollars):
        if dollars in self.prices:
            return True
        
        for price in self.prices:
            b_price = dollars - price
            if b_price in self.prices:
                return True
            
        return False

    def find_first_laptop_more_expensive(self, price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1         
        
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            cur_price = self.rows_by_price[range_middle][-1]
            if cur_price > price:                           
                range_end = range_middle              
            else:                                          
                range_start = range_middle + 1
                        
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1     
        
        return range_start
    
# test budget method
laptops = inventory('laptops.csv')

index = laptops.find_first_laptop_more_expensive(1000) # answer: 683
print("Expected Index: 683", "Returned Index:", index)

index = laptops.find_first_laptop_more_expensive(10000) # answer: 683
print("Expected Index: -1", "Returned Index:", index)

NameError: name 'a_price' is not defined